In [390]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
merged = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
merged.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [391]:
# Calculate the total number of unique schools
school_count = merged["school_name"].nunique()
school_count

15

In [392]:
# Calculate the total number of students
student_count = merged["student_name"].count()
student_count


39170

In [393]:
# Calculate the total budget
total_budget = merged["budget"].sum()
total_budget

82932329558

In [394]:
# Calculate the average (mean) math score
average_math_score = merged["math_score"].mean() 
average_math_score

78.98537145774827

In [395]:
# Calculate the average (mean) reading score
average_reading_score = merged["reading_score"].mean() 
average_reading_score

81.87784018381414

In [396]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = merged[(merged["math_score"] >= 70)].count()["student_name"] #Placing the ["student_name"] after count, narrows the results to only show us for that column.
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage


74.9808526933878

In [397]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = merged[(merged["reading_score"]>=70)].count()["student_name"]
passing_reading_percentage = passing_reading_count/float(student_count)*100
passing_reading_percentage

85.80546336482001

In [398]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = merged[(merged["math_score"] >= 70) & (merged["reading_score"] >= 70)
                                   ].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [399]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame([{"Total Schools":school_count,"Total Students":student_count, "Total Budget":total_budget,
                           "Average Math Score":average_math_score,"Average Reading Score":average_reading_score,
                           "% Passing Math":passing_math_percentage,"%Passing Reading":passing_reading_percentage,
                           "%Overall Passing":overall_passing_rate}])
##using a dictionary is efficient here.

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,%Passing Reading,%Overall Passing
0,15,"39,170","$82,932,329,558.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [400]:
# Use the code provided to select all of the school types
school_types = school_data.set_index(["school_name"])["type"]
school_types

school_name
Huang High School        District
Figueroa High School     District
Shelton High School       Charter
Hernandez High School    District
Griffin High School       Charter
Wilson High School        Charter
Cabrera High School       Charter
Bailey High School       District
Holden High School        Charter
Pena High School          Charter
Wright High School        Charter
Rodriguez High School    District
Johnson High School      District
Ford High School         District
Thomas High School        Charter
Name: type, dtype: object

In [401]:
# Calculate the total student count per school
##This can be extracted using value_counts as we are measuring how many students(each row)
#in each school(value in school_name column)
per_school_counts = merged["school_name"].value_counts()
per_school_counts

school_name
Bailey High School       4976
Johnson High School      4761
Hernandez High School    4635
Rodriguez High School    3999
Figueroa High School     2949
Huang High School        2917
Ford High School         2739
Wilson High School       2283
Cabrera High School      1858
Wright High School       1800
Shelton High School      1761
Thomas High School       1635
Griffin High School      1468
Pena High School          962
Holden High School        427
Name: count, dtype: int64

In [402]:
# Calculate the total school budget and per capita spending per school


per_school_budget = school_data[["school_name","budget"]]
per_school_budget

per_school_budget = merged.groupby(["school_name"])["budget"].mean()
per_school_budget

per_school_capita = per_school_budget / per_school_counts
per_school_capita

school_name
Bailey High School       628.0
Cabrera High School      582.0
Figueroa High School     639.0
Ford High School         644.0
Griffin High School      625.0
Hernandez High School    652.0
Holden High School       581.0
Huang High School        655.0
Johnson High School      650.0
Pena High School         609.0
Rodriguez High School    637.0
Shelton High School      600.0
Thomas High School       638.0
Wilson High School       578.0
Wright High School       583.0
dtype: float64

In [403]:
# Calculate the average test scores per school
per_school_math = merged.groupby(["school_name"])["math_score"].mean()
per_school_math

per_school_reading = merged.groupby(["school_name"])["reading_score"].mean()
per_school_reading

school_name
Bailey High School       81.033963
Cabrera High School      83.975780
Figueroa High School     81.158020
Ford High School         80.746258
Griffin High School      83.816757
Hernandez High School    80.934412
Holden High School       83.814988
Huang High School        81.182722
Johnson High School      80.966394
Pena High School         84.044699
Rodriguez High School    80.744686
Shelton High School      83.725724
Thomas High School       83.848930
Wilson High School       83.989488
Wright High School       83.955000
Name: reading_score, dtype: float64

In [426]:
# Calculate the number of students per school with math scores of 70 or higher


school_students_passing_math = merged[merged["math_score"] >= 70].groupby("school_name")["student_name"].count()
school_students_passing_math


students_passing_math = school_students_passing_math.sum()
students_passing_math




29370

In [427]:
# Calculate the number of students per school with reading scores of 70 or higher
school_students_passing_reading = merged[merged["reading_score"]>=70].groupby("school_name")["student_name"].count()
school_students_passing_reading

students_passing_reading = school_students_passing_reading.sum()
students_passing_reading


33610

In [406]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = merged[(merged["reading_score"] >= 70) & (merged["math_score"] >= 70)].groupby("school_name")["student_name"].count()
students_passing_math_and_reading

school_students_passing_math_and_reading = students_passing_math_and_reading.sum()
school_students_passing_math_and_reading

25528

In [407]:
# Use the provided code to calculate the passing rates
per_school_passing_math = school_students_passing_math / per_school_counts * 100
per_school_passing_math
per_school_passing_reading = school_students_passing_reading / per_school_counts * 100
per_school_passing_reading
overall_passing_rate = students_passing_math_and_reading / per_school_counts * 100
overall_passing_rate

school_name
Bailey High School       54.642283
Cabrera High School      91.334769
Figueroa High School     53.204476
Ford High School         54.289887
Griffin High School      90.599455
Hernandez High School    53.527508
Holden High School       89.227166
Huang High School        53.513884
Johnson High School      53.539172
Pena High School         90.540541
Rodriguez High School    52.988247
Shelton High School      89.892107
Thomas High School       90.948012
Wilson High School       90.582567
Wright High School       90.333333
dtype: float64

In [408]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.DataFrame(merged.groupby("school_name")["student_name"].count())
per_school_summary["School Type"]= school_types
per_school_summary["Average Math Score"]= per_school_math
per_school_summary["Average Reading Score"]= per_school_reading


per_school_summary = per_school_summary.rename(columns={"student_name":"Total Students"})

per_school_summary["Total School Budget"]= per_school_budget
per_school_summary["Per Student Budget"]= per_school_capita                    

per_school_summary["% Passing Math"] = per_school_passing_math
per_school_summary["% Passing Reading"] = per_school_passing_reading
per_school_summary["% Overall Passing"] = overall_passing_rate
# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary.head()

,Total Students,School Type,Average Math Score,Average Reading Score,Total School Budget,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,4976,District,77.048432,81.033963,"$3,124,928.00",$628.00,66.680064,81.933280,54.642283
Cabrera High School,1858,Charter,83.061895,83.975780,"$1,081,356.00",$582.00,94.133477,97.039828,91.334769
Figueroa High School,2949,District,76.711767,81.158020,"$1,884,411.00",$639.00,65.988471,80.739234,53.204476
Ford High School,2739,District,77.102592,80.746258,"$1,763,916.00",$644.00,68.309602,79.299014,54.289887
Griffin High School,1468,Charter,83.351499,83.816757,"$917,500.00",$625.00,93.392371,97.138965,90.599455


## Highest-Performing Schools (by % Overall Passing)

In [409]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values("% Overall Passing", ascending=False)
top_schools.head(5)

,Total Students,School Type,Average Math Score,Average Reading Score,Total School Budget,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,1858,Charter,83.061895,83.975780,"$1,081,356.00",$582.00,94.133477,97.039828,91.334769
Thomas High School,1635,Charter,83.418349,83.848930,"$1,043,130.00",$638.00,93.272171,97.308869,90.948012
Griffin High School,1468,Charter,83.351499,83.816757,"$917,500.00",$625.00,93.392371,97.138965,90.599455
Wilson High School,2283,Charter,83.274201,83.989488,"$1,319,574.00",$578.00,93.867718,96.539641,90.582567
Pena High School,962,Charter,83.839917,84.044699,"$585,858.00",$609.00,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [410]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values("% Overall Passing",ascending=True)
bottom_schools.head(5)

,Total Students,School Type,Average Math Score,Average Reading Score,Total School Budget,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,3999,District,76.842711,80.744686,"$2,547,363.00",$637.00,66.366592,80.220055,52.988247
Figueroa High School,2949,District,76.711767,81.158020,"$1,884,411.00",$639.00,65.988471,80.739234,53.204476
Huang High School,2917,District,76.629414,81.182722,"$1,910,635.00",$655.00,65.683922,81.316421,53.513884
Hernandez High School,4635,District,77.289752,80.934412,"$3,022,020.00",$652.00,66.752967,80.862999,53.527508
Johnson High School,4761,District,77.072464,80.966394,"$3,094,650.00",$650.00,66.057551,81.222432,53.539172


## Math Scores by Grade

In [411]:
# Use the code provided to separate the data by grade
ninth_graders = merged[(merged["grade"] == "9th")]
tenth_graders = merged[(merged["grade"] == "10th")]
eleventh_graders = merged[(merged["grade"] == "11th")]
twelfth_graders = merged[(merged["grade"] == "12th")]


# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_math_scores = merged[merged["grade"]=="9th"].groupby("school_name")["math_score"].mean()
ninth_grade_math_scores

tenth_grader_math_scores = merged[merged["grade"]=="10th"].groupby("school_name")["math_score"].mean()
tenth_grader_math_scores
                                  
eleventh_grader_math_scores = merged[merged["grade"]=="11th"].groupby("school_name")["math_score"].mean()
eleventh_grader_math_scores

twelfth_grader_math_scores = merged[merged["grade"]=="12th"].groupby("school_name")["math_score"].mean()
twelfth_grader_math_scores


# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame(merged.groupby("school_name")['student_name'].count())
math_scores_by_grade = math_scores_by_grade.rename(columns={"student_name":"Total Students"})                               

math_scores_by_grade["9th Grade Math Average"]= ninth_grade_math_scores 
math_scores_by_grade["10th Grade Math Average"]= tenth_grader_math_scores 
math_scores_by_grade["11th Grade Math Average"]= eleventh_grader_math_scores 
math_scores_by_grade["12th Grade Math Average"]= twelfth_grader_math_scores 



# Minor data wrangling
math_scores_by_grade = math_scores_by_grade[["9th Grade Math Average", 
                                                   "10th Grade Math Average", 
                                                   "11th Grade Math Average", 
                                                   "12th Grade Math Average"]]
math_scores_by_grade.index.name = None


# Display the DataFrame
math_scores_by_grade


#The mean math scores across all schools, by grade!
#78.93565918653576 Grade.9 Math Scores
#78.94148308418568 Grade.10 Math Scores
#79.08354822073234 Grade.11 Math Scores
#78.99316369160653 Grade.12 Math Scores

,9th Grade Math Average,10th Grade Math Average,11th Grade Math Average,12th Grade Math Average
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

In [412]:
# Use the code provided to separate the data by grade
ninth_graders = merged[(merged["grade"] == "9th")]
tenth_graders = merged[(merged["grade"] == "10th")]
eleventh_graders = merged[(merged["grade"] == "11th")]
twelfth_graders = merged[(merged["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grade_reading_scores = merged[merged["grade"]=="9th"].groupby("school_name")["reading_score"].mean()
ninth_grade_reading_scores

tenth_grader_reading_scores = merged[merged["grade"]=="10th"].groupby("school_name")["reading_score"].mean()
tenth_grader_reading_scores

eleventh_grader_reading_scores = merged[merged["grade"]=="11th"].groupby("school_name")["reading_score"].mean()
eleventh_grader_reading_scores

twelfth_grader_reading_scores = merged[merged["grade"]=="12th"].groupby("school_name")["reading_score"].mean()
twelfth_grader_reading_scores

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame(merged.groupby("school_name")['student_name'].count())
reading_scores_by_grade = reading_scores_by_grade.rename(columns={"student_name":"Total Students"})

reading_scores_by_grade["9th Grade Reading Average"] = ninth_grade_reading_scores
reading_scores_by_grade["10th Grade Reading Average"] = tenth_grader_reading_scores
reading_scores_by_grade["11th Grade Reading Average"] = eleventh_grader_reading_scores
reading_scores_by_grade["12th Grade Reading Average"] = twelfth_grader_reading_scores


# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th Grade Reading Average", 
                                                   "10th Grade Reading Average", 
                                                   "11th Grade Reading Average", 
                                                   "12th Grade Reading Average"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

,9th Grade Reading Average,10th Grade Reading Average,11th Grade Reading Average,12th Grade Reading Average
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [413]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [414]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()


In [415]:
# Use `pd.cut` to categorize spending based on the bins.
pd.cut(per_school_capita,spending_bins, labels=labels)

school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita ,spending_bins, labels=labels)
school_spending_df

,Total Students,School Type,Average Math Score,Average Reading Score,Total School Budget,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
school_name,,,,,,,,,,
Bailey High School,4976,District,77.048432,81.033963,"$3,124,928.00",$628.00,66.680064,81.933280,54.642283,$585-630
Cabrera High School,1858,Charter,83.061895,83.975780,"$1,081,356.00",$582.00,94.133477,97.039828,91.334769,<$585
Figueroa High School,2949,District,76.711767,81.158020,"$1,884,411.00",$639.00,65.988471,80.739234,53.204476,$630-645
Ford High School,2739,District,77.102592,80.746258,"$1,763,916.00",$644.00,68.309602,79.299014,54.289887,$630-645
Griffin High School,1468,Charter,83.351499,83.816757,"$917,500.00",$625.00,93.392371,97.138965,90.599455,$585-630
Hernandez High School,4635,District,77.289752,80.934412,"$3,022,020.00",$652.00,66.752967,80.862999,53.527508,$645-680
Holden High School,427,Charter,83.803279,83.814988,"$248,087.00",$581.00,92.505855,96.252927,89.227166,<$585
Huang High School,2917,District,76.629414,81.182722,"$1,910,635.00",$655.00,65.683922,81.316421,53.513884,$645-680
Johnson High School,4761,District,77.072464,80.966394,"$3,094,650.00",$650.00,66.057551,81.222432,53.539172,$645-680


In [416]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

In [417]:
# Assemble into DataFrame
spending_summary = pd.DataFrame(school_spending_df.groupby("Spending Ranges (Per Student)")["Total Students"].count())

spending_summary["Average Math Score"] = spending_math_scores
spending_summary["Average Reading Score"] = spending_reading_scores 
spending_summary["% Passing Math"] = spending_passing_math
spending_summary["% Passing Reading"] = spending_passing_reading
spending_summary["% Overall Passing"] = overall_passing_spending

spending_summary = spending_summary[["Average Math Score","Average Reading Score",
                                    "% Passing Math","% Passing Reading","% Overall Passing"]]

# Display results
spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

In [418]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [419]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.
pd.cut(per_school_summary["Total Students"], size_bins, labels=labels)

per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"], size_bins, labels=labels)
per_school_summary

,Total Students,School Type,Average Math Score,Average Reading Score,Total School Budget,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing,School Size
school_name,,,,,,,,,,
Bailey High School,4976,District,77.048432,81.033963,"$3,124,928.00",$628.00,66.680064,81.933280,54.642283,Large (2000-5000)
Cabrera High School,1858,Charter,83.061895,83.975780,"$1,081,356.00",$582.00,94.133477,97.039828,91.334769,Medium (1000-2000)
Figueroa High School,2949,District,76.711767,81.158020,"$1,884,411.00",$639.00,65.988471,80.739234,53.204476,Large (2000-5000)
Ford High School,2739,District,77.102592,80.746258,"$1,763,916.00",$644.00,68.309602,79.299014,54.289887,Large (2000-5000)
Griffin High School,1468,Charter,83.351499,83.816757,"$917,500.00",$625.00,93.392371,97.138965,90.599455,Medium (1000-2000)
Hernandez High School,4635,District,77.289752,80.934412,"$3,022,020.00",$652.00,66.752967,80.862999,53.527508,Large (2000-5000)
Holden High School,427,Charter,83.803279,83.814988,"$248,087.00",$581.00,92.505855,96.252927,89.227166,Small (<1000)
Huang High School,2917,District,76.629414,81.182722,"$1,910,635.00",$655.00,65.683922,81.316421,53.513884,Large (2000-5000)
Johnson High School,4761,District,77.072464,80.966394,"$3,094,650.00",$650.00,66.057551,81.222432,53.539172,Large (2000-5000)


In [420]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

In [421]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame(per_school_summary.groupby("School Size")["Total Students"].count())

size_summary["Average Math Score"] = size_math_scores
size_summary["Average Reading Score"] = size_reading_scores                 
size_summary["% Passing Math"] = size_passing_math
size_summary["% Passing Reading"] = size_passing_reading
size_summary["% Overall Passing"] = size_overall_passing  

size_summary = size_summary[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]]
size_summary.index.name = None
# Display results
size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

In [422]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

In [423]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame(per_school_summary.groupby("School Type")["Total Students"].count())

type_summary["Average Math Score"] = average_math_score_by_type
type_summary["Average Reading Score"] = average_reading_score_by_type
type_summary["% Passing Math"] = average_percent_passing_math_by_type
type_summary["% Passing Reading"] = average_percent_passing_reading_by_type
type_summary["% Overall Passing"] = average_percent_overall_passing_by_type

type_summary = type_summary[["Average Math Score","Average Reading Score",
                             "% Passing Math","% Passing Reading","% Overall Passing"]]
# Display results
type_summary




,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
